## Predict portal-to-central vein axes
**Input**: Portal & Central veins ; Periportal & Central Regions\
**Mechanism**: Contraint method: Candidate axes must follow a set of rules\
**Output**: Arrays with start- and endpoints of predicted portal-to-central vein axes

In [1]:
# Libraries
import os, yaml
import dask.array as da
import spatialdata as sd
from skimage.segmentation import find_boundaries
import numpy as np
from skimage.measure import label, regionprops
from functions.axis_prediction_utils import *

### Parameters

In [3]:
#Paths
sample_type = 'Flox_Control'
data_dir = '/srv/data/michielvc/data/CSF1_ligand_data/SpatialData/'

# Open config file and extract relevant information
config = yaml.safe_load(open(data_dir + f'{sample_type}_config.yaml'))
file_names = config['file_names']

# Parameters
glul_ch = config['marker_channel']['Glul']
ecadh_ch = config['marker_channel']['Ecadh']

central_zone_label = config['zonation_labels']['central_zone']
periportal_zone_label = config['zonation_labels']['periportal_zone']

central_vein_label = config['segment_labels']['central_vein']
portal_vein_label = config['segment_labels']['portal_vein']

max_axis_length = config['axis_properties']['max_length']
min_axis_length = config['axis_properties']['min_length']


### Load SpatialData object

In [4]:
sdata = sd.read_zarr(os.path.join(data_dir, f'{sample_type}.sd'))
sdata

SpatialData object with:
├── Images
│     ├── 'E2M1FloxControl_preprocessed': SpatialImage[cyx] (5, 7533, 16659)
│     ├── 'E2M1FloxControl_raw': SpatialImage[cyx] (5, 7533, 16659)
│     ├── 'E2M3FloxControl_preprocessed': SpatialImage[cyx] (5, 11178, 18468)
│     ├── 'E2M3FloxControl_raw': SpatialImage[cyx] (5, 11178, 18468)
│     ├── 'E2M5FloxControl_preprocessed': SpatialImage[cyx] (5, 9342, 12987)
│     ├── 'E2M5FloxControl_raw': SpatialImage[cyx] (5, 9342, 12987)
│     ├── 'E2M6FloxControl_preprocessed': SpatialImage[cyx] (5, 11205, 14823)
│     ├── 'E2M6FloxControl_raw': SpatialImage[cyx] (5, 11205, 14823)
│     ├── 'E2M7FloxControl_preprocessed': SpatialImage[cyx] (5, 9342, 16632)
│     ├── 'E2M7FloxControl_raw': SpatialImage[cyx] (5, 9342, 16632)
│     ├── 'E2M8FloxControl_preprocessed': SpatialImage[cyx] (5, 7533, 14823)
│     ├── 'E2M8FloxControl_raw': SpatialImage[cyx] (5, 7533, 14823)
│     ├── 'E2M15FloxControl_preprocessed': SpatialImage[cyx] (5, 9342, 14823)
│     ├── 'E

### Calculate P-C axes  

dict_of_neighbours = {c_label:[
 np.array([p_label1, (p1, p2)1, dist1])
 np.array([p_label2, (p1, p2)2, dist2])
 ...
 ]}

In [5]:
# Read veins and regions
file_name = file_names[0]
vein_segments = sdata.labels[file_name + '_vein_segments']
zonation_segments = sdata.labels[file_name + '_zonation']

In [6]:
# Process data

# Get zones where vein segments overlap with region segments and remove overlapping zones from veins
overlapping_zones = da.logical_and(vein_segments, zonation_segments)
shunk_vein_segments = np.asarray(vein_segments, dtype=np.uint8)  # Out of dask! TODO
shunk_vein_segments[overlapping_zones] = 0

# get labeled portal and central vein segments boundaries
portal_vein_segments = shunk_vein_segments == portal_vein_label
central_vein_segments = shunk_vein_segments == central_vein_label

# get labeled central and periportal zones boundaries
c_labels = label(find_boundaries(central_vein_segments))
p_labels = label(find_boundaries(portal_vein_segments))
nb_c_veins = np.max(c_labels)

image_dimensions = c_labels.shape

In [7]:
# for c_props in regionprops(c_labels):
#     c_label = c_props.label
c_props = regionprops(c_labels)[0]
c_label = c_props.label
c_border_coordinates = c_props.coords

# isolate zone around current central vein in which potential partner portal veins can be found (limited by max_axis_length)
zone_y1, zone_y2, zone_x1, zone_x2 = frame_around_object(c_props, size_increase=max_axis_length, im_dimensions=image_dimensions)
portal_veins_in_zone = p_labels[zone_y1:zone_y2, zone_x1:zone_x2]
pv_in_zone_props = regionprops(portal_veins_in_zone)

# assess for each portal vein in zone if it is a partner vein
# for p_props in pv_in_zone_props:
p_props = pv_in_zone_props[0]
p_border_coordinates = p_props.coords

# calculate shortest axis between current central vein and current portal vein
axis, length = get_shortest_axis(c_border_coordinates, p_border_coordinates)
# if distance < min_axis_length:
#     continue



In [9]:
axis,length

((array([1320, 5977]), array([1395, 1230])), 4747.592442491246)